In [43]:
MODEL = "gpt-4o"
#MODEL = "gpt-4o-mini"
#MODEL = "gpt-3.5-turbo"
title = "AI"

from openai import OpenAI
import markdown
import os
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()

In [75]:
# Function to create a clickable element from headlines
def process_headlines(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Find all h1 text
    for h in soup.find_all(['h1', 'h2', 'h3']):
        h_text = h.text.strip()

        # Create a clickable span with a custom style and onclick event
        new_tag = soup.new_tag("span", style="color: blue; cursor: pointer;", onclick=f"gotoContent('{h_text}')")
        new_tag.string = h_text
        
        # Clear the content of the header and append the span inside it
        h.clear()
        h.append(new_tag)

    return str(soup)

In [22]:
# Function to create a hyperlink or clickable element from bold text based on whether a file exists
def process_bold_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Find all bold text (both <strong> and <b> tags)
    for bold_tag in soup.find_all(['strong', 'b']):
        bold_text = bold_tag.text.strip()
        html_file_name = f"{bold_text}.html"

        # Check if a file with this name exists
        if os.path.exists(html_file_name):
            # Replace bold text with a hyperlink
            new_tag = soup.new_tag("a", href=html_file_name)
            new_tag.string = bold_text
            bold_tag.replace_with(new_tag)
        else:
            # Create a clickable span with a custom style and onclick event
            new_tag = soup.new_tag("span", style="color: blue; cursor: pointer;", onclick=f"fetchTitle('{bold_text}')")
            new_tag.string = bold_text
            bold_tag.replace_with(new_tag)

    return str(soup)

In [5]:
completion = client.chat.completions.create(
    messages = [{"role": "user", "content": title}],
    model = MODEL,
    )
content = completion.choices[0].message.content
html_content = markdown.markdown(content)

# Creates a clickable element from each instance of bold text in html_content. If user clicks on the bold text, a function check whether there is html document with the same name as the bold text. If there is, user is redirected to this html document. If not, OpenAI API is called with messages = [{"role": "user", "content": "title"}], where title is content of the bold text.

In [17]:
d = completion.to_dict()
df = pd.DataFrame([d])
df.to_json(title)

In [77]:
html_content = markdown.markdown(content, extensions=['markdown.extensions.extra', 'markdown.extensions.nl2br', 'markdown.extensions.sane_lists'])
html_content = process_headlines(html_content)

In [79]:
# Wrapping the content in basic HTML structure
html_page = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{title}</title>
    <script src="scripts.js"></script>
</head>
<body>
    <p><h1>{title}</h1></p>
    <p>{html_content}</p>
</body>
</html>
"""

# Optionally, you can save this HTML content to a file:
with open(f"{title}.html", "w") as file:
    file.write(html_page)

print("HTML content created successfully.")


HTML content created successfully.
